In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from email import message_from_file
from email.parser import BytesParser
from email import policy
import email

In [ ]:
def read_emails(folder_path):
  emails=[]
  for file_name in os.listdir(folder_path):
    file_path=os.path.join(folder_path,file_name)
    with open(file_path, 'r', encoding="utf-8", errors="ignore") as file:
      content= file.read()
      message=message_from_file(file)
      emails.append({"filename":file_name, "content":content, "label":None})
  return emails

In [ ]:
folder_path="/content/drive/MyDrive/hard_ham"
ham_emails=read_emails(folder_path)

In [ ]:
print(len(ham_emails))

250


In [ ]:
print(type(ham_emails))

<class 'list'>


In [ ]:
folder_path_3="/content/drive/MyDrive/easy_ham_2"
ham_emails_2=read_emails(folder_path_3)

In [ ]:
for i in range(len(ham_emails_2)):
  ham_emails.append(ham_emails_2[i])

In [ ]:
print(len(ham_emails))

2834


In [ ]:
for i in range(len(ham_emails)):
  ham_emails[i]["label"]=1

In [ ]:
folder_path_2="/content/drive/MyDrive/spam"
spam_emails=read_emails(folder_path_2)

In [ ]:
print(len(spam_emails))

501


In [ ]:
folder_path_4="/content/drive/MyDrive/spam_2"
spam_emails_2=read_emails(folder_path_4)

In [ ]:
for i in range(len(spam_emails_2)):
  spam_emails.append(spam_emails_2[i])

In [ ]:
print(len(spam_emails))

1898


In [ ]:
def email_parser(content):
  email.parser.BytesParser(policy=email.policy.default).parse(content)

In [ ]:
def email_body(message):
  body=""
  if message.is_multipart():
    body += "~@"
    for part in message.walk():
      if part.get_content_type()=="text/plain":
        body += part.get_payload()

  else:
    body=message.get_payload()

  return(body)

In [ ]:
def email_struct(content):
  struct={}

  message=BytesParser(policy=policy.default).parsebytes(
      content.encode("utf-8", errors="ignore")
  )

  struct["filename"]=message.get("filename", "")
  struct["subject"]=message.get("subject","")
  struct["sender"]=message.get("from","")
  struct["date"]=message.get("date","")
  struct["body"]=email_body(message)

  return(struct)

In [ ]:
count=0
for i in ham_emails:
  ham_struct=email_struct(i["content"])
  # print(ham_struct["body"][0:3])
  if(ham_struct["body"][0:2]=="~@"):
    count+=1

print(count*100/len(ham_emails))

5.081157374735357


In [ ]:
count=0
for i in spam_emails:
  spam_struct=email_struct(i["content"])
  if(spam_struct["body"][0:2]=="~@"):
    count+=1

print(count*100/len(spam_emails))

16.0695468914647


In [ ]:
import re

def extract_email_body(email_text):
    # Define patterns for common separators between headers and body
    separators = [
        re.compile(r"\n\n", re.MULTILINE),      # Double newline
        re.compile(r"\n\s*\n", re.MULTILINE),   # Double newline with optional spaces
        re.compile(r"\n\r\n", re.MULTILINE),    # Double newline with carriage return
    ]

    # Try to find the first occurrence of a separator
    for separator_pattern in separators:
        match = separator_pattern.search(email_text)
        if match:
            body_start = match.end()
            return email_text[body_start:].strip()

    # If no separator is found, return the entire email text
    return email_text.strip()


# body_content = extract_email_body(email_text)
# print(body_content)


In [ ]:
print(spam_emails[0].keys())

dict_keys(['filename', 'content', 'label'])


In [ ]:
df=pd.DataFrame(columns=spam_emails[0].keys())
df

,filename,content,label


In [ ]:
print(len(df.index))

0


In [ ]:
for i in range (len(spam_emails)):
  df.loc[len(df.index)]=spam_emails[i]

df

,filename,content,label
0,0225.7082ef8585280a42940bf98f9be50e55,From barristerbassey@arabia.com Wed Sep 4 11...,None
1,0022.4b5cf3c16feb88dd6932a8c46a41946c,From ilug-admin@linux.ie Fri Aug 23 11:08:03 ...,None
2,0214.b5ba0ff48cee07a36c6f312de7f77207,From none@none.com Mon Sep 2 16:28:01 2002\n...,None
3,0015.1b871d654560011a0aaa29bb4e9054f7,From approvals@mindspring.com Fri Aug 23 11:0...,None
4,0122.21b041c1ad2be417102d7f5d3f0b7045,From jani@topmail.dk Tue Aug 27 08:32:58 2002...,None
...,...,...,...
1893,01396.e80a10644810bc2ae3c1b58c5fd38dfa,From Professional_Career_Development_Institute...,None
1894,01395.cb33d1d72f42e4ab9268729917bf428b,From johnson11@aa64.toplinequotes.com Wed Dec...,None
1895,01399.2319643317e2c5193d574e40a71809c2,From cweqx@dialix.oz.au Tue Aug 6 11:03:54 2...,None
1896,01397.f75f0dd0dd923faefa3e9cc5ecb8c906,From tba@insiq.us Wed Dec 4 11:46:34 2002\nR...,None


In [ ]:
for i in ham_emails:
  df.loc[len(df.index)]=i

df

,filename,content,label
0,0225.7082ef8585280a42940bf98f9be50e55,From barristerbassey@arabia.com Wed Sep 4 11...,None
1,0022.4b5cf3c16feb88dd6932a8c46a41946c,From ilug-admin@linux.ie Fri Aug 23 11:08:03 ...,None
2,0214.b5ba0ff48cee07a36c6f312de7f77207,From none@none.com Mon Sep 2 16:28:01 2002\n...,None
3,0015.1b871d654560011a0aaa29bb4e9054f7,From approvals@mindspring.com Fri Aug 23 11:0...,None
4,0122.21b041c1ad2be417102d7f5d3f0b7045,From jani@topmail.dk Tue Aug 27 08:32:58 2002...,None
...,...,...,...
4727,01356.8d72d21568fbfdd4aec060fa8826832a,From nobody@sonic.spamtraps.taint.org Wed Aug...,1
4728,01360.12e20ec23520b1c5515424bf77ed94c8,From vulnwatch-return-430-jm=jmason.org@vulnwa...,1
4729,01357.102c99f3c387520d97c19c884c886355,From spamassassin-talk-admin@lists.sourceforge...,1
4730,01359.8252955a19112d1adb6abeef20ffb9ea,From jm@webnote.net Fri Aug 9 15:12:40 2002\...,1


In [ ]:
df.isnull().sum()

filename       0
content        0
label       1898
dtype: int64

In [ ]:
df["label"]=df["label"].fillna(0)

In [ ]:
for i in range (1,len(df.index)+1):
  # print(i)
  df.iloc[i-1,[0]]=("Email{0}".format(i))

df

,filename,content,label
0,Email1,From barristerbassey@arabia.com Wed Sep 4 11...,0
1,Email2,From ilug-admin@linux.ie Fri Aug 23 11:08:03 ...,0
2,Email3,From none@none.com Mon Sep 2 16:28:01 2002\n...,0
3,Email4,From approvals@mindspring.com Fri Aug 23 11:0...,0
4,Email5,From jani@topmail.dk Tue Aug 27 08:32:58 2002...,0
...,...,...,...
4727,Email4728,From nobody@sonic.spamtraps.taint.org Wed Aug...,1
4728,Email4729,From vulnwatch-return-430-jm=jmason.org@vulnwa...,1
4729,Email4730,From spamassassin-talk-admin@lists.sourceforge...,1
4730,Email4731,From jm@webnote.net Fri Aug 9 15:12:40 2002\...,1


In [ ]:
for i in range (0,len(spam_emails)):
  temp=email_struct(spam_emails[i]["content"])
  spam_emails[i]["content"]=temp["body"]

for i in range (0,len(ham_emails)):
  temp=email_struct(ham_emails[i]["content"])
  ham_emails[i]["content"]=temp["body"]

In [ ]:
for i in range(len(df.index)):
  if(i<len(spam_emails)):
    body=spam_emails[i]["content"]
    df.at [i,"content"]=body
  else:
    body=ham_emails[i-len(spam_emails)]["content"]
    df.at [i,"content"]=body

In [ ]:
df

,filename,content,label
0,Email1,"~@BASSEY& ASSOCIATES\nFALOMO, IKOYI.\nLAGOS - ...",0
1,Email2,CENTRAL BANK OF NIGERIA\nFOREIGN REMITTANCE DE...,0
2,Email3,"<p>Dear user,</p>\n<p>CyberAge Dating Club is ...",0
3,Email4,==============================================...,0
4,Email5,~@SGVscCB3YW50ZWQuICBXZSBhcmUgYSAxNCB5ZWFyIG9s...,0
...,...,...,...
4727,Email4728,Problem with spamtrap\nCould not lock /home/yy...,1
4728,Email4729,Macromedia Shockwave Flash Malformed Header Ov...,1
4729,Email4730,"""Tony L. Svanstrom"" <tony@svanstrom.com> write...",1
4730,Email4731,"\nTheo Van Dinter said:\n\n> Well, I have my u...",1


In [ ]:
# Converting the content to lower case in order to unify.

df["content"]=df["content"].str.lower()

In [ ]:
# Removing all the HTML tags using regular expression

import re
def remove_html_tags(text):
  pattern= re.compile("<.*?>")
  return pattern.sub(r'', text)

In [ ]:
df["content"]=df["content"].apply(remove_html_tags)

In [ ]:
# function to remove url

def remove_url(text):
  pattern=re.compile(r"https?://\S+|www\.\S+")
  return pattern.sub(r'', text)

In [ ]:
df["content"]=df["content"].apply(remove_url)

In [ ]:
import string

punctuation=string.punctuation
print(type(punctuation))

<class 'str'>


In [ ]:
# Function to remove punctuation to faciliate feature engineering

def remove_punc(text):
  return text.translate(str.maketrans("", "", punctuation))

In [ ]:
df["content"]=df["content"].apply(remove_punc)

In [ ]:
from textblob import TextBlob

In [ ]:
# function to correct spelling mistakes

def remove_spell_mistake(text):
  textBlb=TextBlob(text)
  textBlb.correct().string
  return textBlb

In [ ]:
df["content"]=df["content"].apply(remove_spell_mistake)

KeyboardInterrupt: ignored

In [ ]:
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
# function to remove stopwords(neutral word that do not provide any info about the content)

def remove_stopwords(text):
  new_text=[]

  for word in text.split():
    if word in stopwords.words("english"):
      new_text.append("")
    else:
      new_text.append(word)
  x=new_text[:]
  new_text.clear()
  return " ".join(x)

In [ ]:
df["content"]=df["content"].apply(remove_stopwords)

In [ ]:
# function to remove emojis

def remove_emoji(text):
  emoji_pattern=re.compile("["
                u"\U0001F600-\U0001F64F"
                u"\U0001F300-\U0001F5FF"
                u"\U0001F680-\U0001F6FF"
                u"\U0001F1E0-\U0001F1FF"
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r"", text)

In [ ]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
import emoji

In [ ]:
# function to replace emojis by the meaning

def replace_emoji(text):
  return emoji.demojize(text)

In [ ]:
df["content"]=df["content"].apply(replace_emoji)

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
chat_words= {
    "lol": "laugh out loud",
    "brb": "be right back",
    "gtg": "got to go",
    "btw": "by the way",
    "omg": "oh my god",
    "idk": "I don't know",
    "ttyl": "talk to you later",
    "g2g": "got to go",
    "afk": "away from keyboard",
    "thx": "thanks",
    "np": "no problem",
    "imo": "in my opinion",
    "fyi": "for your information",
    "smh": "shaking my head",
    "bff": "best friends forever",
    "gr8": "great",
    "hmu": "hit me up",
    "tmi": "too much information",
    "imo": "in my opinion",
    "irl": "in real life",
    "rofl": "rolling on the floor laughing",
    "fomo": "fear of missing out",
    "nsfw": "not safe for work",
    "wcw": "woman crush Wednesday",
    "mcm": "man crush Monday",
    "fwiw": "for what it's worth",
    "smh": "shaking my head",
    "hbd": "happy birthday",
    "tbh": "to be honest",
    "dm": "direct message",
    "otp": "on the phone",
    "irl": "in real life",
    "nvm": "never mind",
    "ily": "I love you",
    "omw": "on my way",
    "btw": "by the way",
    "fb": "Facebook",
    "ig": "Instagram",
    "tg": "Telegram",
    "imo": "in my opinion",
    "vip": "very important person",
    "tho": "though",
    "fyi": "for your information",
    "yolo": "you only live once",
    "ftw": "for the win",
    "icymi": "in case you missed it",
    "hth": "hope this helps",
    "ic": "I see",
    "jk": "just kidding",
    "np": "no problem",
    "ppl": "people",
    "rly": "really",
    "srsly": "seriously",
    "tbt": "throwback Thursday",
    "gtfo": "get the f*** out",
    "imho": "in my humble opinion",
    "bruh": "brother",
    "gg": "good game",
    "gl": "good luck",
    "hf": "have fun",
    "tgif": "thank goodness it's Friday",
    "b2b": "business to business",
    "b2c": "business to consumer",
    "omg": "oh my goodness",
    "imo": "in my opinion",
    "oml": "oh my lord",
    "dw": "don't worry",
    "swag": "style and confidence",
    "bday": "birthday",
    "grats": "congratulations",
    "cya": "see you",
    "smexy": "sexy and smart",
    "ilyt": "I love you too",
    "nsfl": "not safe for life",
    "ykwim": "you know what I mean",
    "yolo": "you only live once",
    "ttyt": "talk to you tomorrow",
    "btw": "by the way",
    "oml": "oh my Lord",
    "nbd": "no big deal",
    "sol": "sooner or later",
    "lmao": "laughing my ass off",
    "yw": "you're welcome",
    "irl": "in real life",
    "asap": "as soon as possible",
    "fml": "f*** my life",
    "grats": "congratulations",
    "hth": "hope this helps",
    "ic": "I see",
    "idc": "I don't care",
    "smh": "shake my head",
    "thx": "thanks",
    "wtf": "what the f***",
    "omw": "on my way",
    "tbh": "to be honest",
    "irl": "in real life",
    "imo": "in my opinion",
    "hbu": "how about you",
    "imo": "in my opinion",
    "sry": "sorry",
}

In [ ]:
def remove_chat_words(text):
  new_text=[chat_words[i] if i in chat_words.keys() else i for i in text.split()]
  return " ".join(new_text)

In [ ]:
df["content"]=df["content"].apply(remove_chat_words)

In [ ]:
# stemming using stem_words
ps=PorterStemmer()
def stem_words(text):
  return " ".join([ps.stem(word) for word in text.split()])

In [ ]:
df["content"]=df["content"].apply(stem_words)

In [ ]:
# function to replace numbers

def replace_numbers(text):
  new_text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', text)
  return new_text

In [ ]:
df["content"]=df["content"].apply(replace_numbers)

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)

In [ ]:
bow= cv.fit_transform(df["content"])

In [ ]:
print(cv.vocabulary_)

{'associ': 172, 'nigeria': 1796, 'kind': 1468, 'attent': 183, 'request': 2240, 'benefit': 249, 'humbl': 1289, 'send': 2367, 'mail': 1592, 'content': 569, 'meet': 1647, 'person': 1983, 'busi': 351, 'ethic': 879, 'apolog': 123, 'advanc': 36, 'william': 2938, 'attorney': 185, 'law': 1500, 'repres': 2238, 'estat': 875, 'chief': 440, 'secur': 2357, 'advic': 39, 'militari': 1671, 'leader': 1510, 'late': 1494, 'gener': 1126, 'abacha': 1, 'die': 718, 'numberth': 1854, 'june': 1448, 'number': 1815, 'new': 1785, 'democrat': 688, 'countri': 602, 'client': 467, 'come': 505, 'sever': 2393, 'due': 784, 'sensit': 2372, 'posit': 2035, 'held': 1231, 'last': 1493, 'present': 2058, 'hous': 1276, 'arrest': 156, 'restrict': 2256, 'main': 1595, 'purpos': 2129, 'propos': 2108, 'might': 1668, 'interest': 1384, 'inform': 1355, 'exist': 914, 'fund': 1105, 'deposit': 694, 'compani': 519, 'came': 365, 'part': 1937, 'vote': 2872, 'presid': 2060, 'happen': 1200, 'transfer': 2752, 'central': 413, 'bank': 219, 'aim':

In [ ]:
word_occurrence=pd.DataFrame(bow.toarray(),columns=cv.get_feature_names_out())
word_occurrence

,aactiv,abacha,abandon,abil,abl,absolut,abus,acceler,accept,access,...,young,your,youv,yr,zdnet,zealot,zero,zip,zone,zoom
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4728,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4729,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4730,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def word_count(word_occurrence):

  word_counts = bow.toarray().sum(axis=0)
  vocabulary=word_occurrence.columns
  res=dict(zip(vocabulary,word_counts))
  return res

In [ ]:
res=word_count(word_occurrence)

In [ ]:
for i,j in res.items():
  print(i,":",j)

aactiv : 55
abacha : 48
abandon : 49
abil : 157
abl : 470
absolut : 269
abus : 100
acceler : 60
accept : 342
access : 781
accessori : 39
accord : 189
account : 672
accur : 39
achiev : 113
acknowledg : 89
acquir : 98
acquisit : 52
across : 163
act : 363
action : 225
activ : 280
actual : 631
ad : 872
adam : 145
adapt : 70
add : 428
addit : 349
address : 2406
adjust : 55
admin : 84
administr : 197
admit : 37
adob : 38
adopt : 59
adult : 143
advanc : 165
advantag : 181
advertis : 645
advic : 125
advis : 89
advisor : 59
advisori : 47
affect : 104
affili : 108
afford : 96
africa : 72
age : 395
agenc : 141
agent : 213
ago : 284
agre : 172
agreeabl : 41
agreement : 76
ah : 49
ahead : 62
ahov : 53
aid : 49
aim : 46
air : 95
airport : 65
al : 136
alan : 55
alarm : 54
album : 76
alert : 86
alfr : 46
algorithm : 85
aligncent : 42
alignnumberdcent : 152
alignnumberdnumbercenternumb : 68
alink : 55
allianc : 38
allow : 599
almost : 271
alon : 104
along : 217
alreadi : 457
also : 1707
alt : 75
alter 

In [ ]:
df.rename(columns={"label":"LABEL"},inplace=True)

In [ ]:
df

,filename,content,LABEL
0,Email1,bassey associ falomo ikoyi lago nigeria kind a...,0
1,Email2,central bank nigeria foreign remitt dept tinub...,0
2,Email3,dear user cyberag date club contact behalf jen...,0
3,Email4,hundr lender compet loan fact interest rate lo...,0
4,Email5,sgvsccbnumberywnumberzwquicbxzsbhcmugysaxncbnu...,0
...,...,...,...
4727,Email4728,problem spamtrap could lock homeyyyylibspamtra...,1
4728,Email4729,macromedia shockwav flash malform header overf...,1
4729,Email4730,toni l svanstrom write in my humbl opinion wro...,1
4730,Email4731,theo van dinter said well user bounc spam mean...,1


In [ ]:
df_final=pd.concat([df, word_occurrence], axis=1)
df_final

,filename,content,LABEL,aactiv,abacha,abandon,abil,abl,absolut,abus,...,young,your,youv,yr,zdnet,zealot,zero,zip,zone,zoom
0,Email1,bassey associ falomo ikoyi lago nigeria kind a...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email2,central bank nigeria foreign remitt dept tinub...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Email3,dear user cyberag date club contact behalf jen...,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,Email4,hundr lender compet loan fact interest rate lo...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Email5,sgvsccbnumberywnumberzwquicbxzsbhcmugysaxncbnu...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,Email4728,problem spamtrap could lock homeyyyylibspamtra...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4728,Email4729,macromedia shockwav flash malform header overf...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4729,Email4730,toni l svanstrom write in my humbl opinion wro...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4730,Email4731,theo van dinter said well user bounc spam mean...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_final.drop(columns=["content"],inplace=True)
df_final

,filename,LABEL,aactiv,abacha,abandon,abil,abl,absolut,abus,acceler,...,young,your,youv,yr,zdnet,zealot,zero,zip,zone,zoom
0,Email1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Email3,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Email4,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Email5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,Email4728,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4728,Email4729,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4729,Email4730,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4730,Email4731,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x=df_final.drop(columns=["filename","LABEL"])
y=df_final["LABEL"]

In [ ]:
y

0       0
1       0
2       0
3       0
4       0
       ..
4727    1
4728    1
4729    1
4730    1
4731    1
Name: LABEL, Length: 4732, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.25, stratify=y)

In [ ]:
y_train.value_counts()

1    2126
0    1423
Name: LABEL, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:
random_class=RandomForestClassifier()
logistic_class=LogisticRegression()

In [ ]:
random_class.fit(x_train,y_train)

RandomForestClassifier()

In [ ]:
cross_val_score(random_class,x_train,y_train,cv=3,scoring="accuracy")

array([0.97633136, 0.96703297, 0.96111581])

In [ ]:
logistic_class.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
cross_val_score(logistic_class,x_train,y_train,cv=3,scoring="accuracy")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

array([0.98393914, 0.96618766, 0.96449704])

In [ ]:
y_pred_random=random_class.predict(x_test)
print("Random Forest accuracy {}".format(accuracy_score(y_test,y_pred_random)))

Random Forest accuracy 0.9873203719357565


In [ ]:
y_pred_logistic=logistic_class.predict(x_test)
print("Logistic Regression accuracy {}".format(accuracy_score(y_test,y_pred_logistic)))

Logistic Regression accuracy 0.9881656804733728
